## mhp ground-up refactor / headers etc. TODO

In [1]:
#%pip install spacy

In [2]:
import re
import pandas as pd
import spacy

In [3]:
cd ..

/Users/sskeen/anaconda_projects/mhp_subtle_discrimination


In [4]:
# import annotated `d_train`

d = pd.read_excel("d_annotated_prelim.xlsx", index_col = [0])

d.info()
d.head(6)

<class 'pandas.core.frame.DataFrame'>
Index: 1005 entries, 0 to 1004
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   EmailPairID      1005 non-null   int64 
 1   WithinPatientID  1005 non-null   int64 
 2   FirstInPair      1005 non-null   object
 3   MHP ID           1005 non-null   object
 4   Unique ID        1005 non-null   object
 5   pilot            1005 non-null   int64 
 6   text             1003 non-null   object
 7   prbl             1005 non-null   object
 8   agnt             1002 non-null   object
 9   afrm             1005 non-null   object
 10  brdn             1004 non-null   object
 11  dmnd             1005 non-null   object
 12  fitt             1004 non-null   object
 13  just             1003 non-null   object
 14  rbnd             1005 non-null   object
 15  refl             1004 non-null   object
 16  rtnl             999 non-null    object
 17  note             1003 non-null   objec

,EmailPairID,WithinPatientID,FirstInPair,MHP ID,Unique ID,pilot,text,prbl,agnt,afrm,brdn,dmnd,fitt,just,rbnd,refl,rtnl,note
/;,,,,,,,,,,,,,,,,,,
0,52,5,Y,1086440,52_5_Y_1086440,0,"Hi <|PII|>, Thanks for reaching out! Unfortuna...",,,,,,1,1,,,<|JUST|> Unfortunately I don’t accept <|PII|> ...,
1,11,4,Y,.,.,1,"Hi <PII>, Thanks for reaching out! What is you...",,,,,,,,,,,
2,316,1,Y,1306860,316_1_Y_1306860,0,"Hello <|PII|>, I received your psychology toda...",,,,,,,,,,,
3,63,5,Y,.,.,1,"Good Morning Darius, I am very sorry to hear y...",,,1,,,1,,,1,"<|REFL|> Good Morning Darius, I am very sorry ...",
4,35,10,Y,.,.,1,Hi. This is <PII>. I am returning a call for <...,,,,,,,,,,,
5,12,6,Y,.,.,1,"Hi <PII>, Thank you for reaching out. Before w...",,,,,,,,,,,


In [5]:
#!python -m spacy download en_core_web_lg --user

In [ ]:
# load spaCy NER pipeline

nlp = spacy.load("en_core_web_lg")

# redact named entities

PII_LABELS = {
    "PERSON", "NORP", "FAC", "ORG", 
    "GPE", "LOC", "PRODUCT", "EVENT",
    }

def redact_named_entities(doc) -> str:
    '''
    Replaces all spaCy-recognized (configurable upstream) 
    PII_LABELS with `<PII>` pseudoword token.
    '''
    chars = list(doc.text)
    for ent in sorted(doc.ents, key = lambda e: e.start_char, reverse = True):
        if ent.label_ not in PII_LABELS:
            continue
        chars[ent.start_char : ent.end_char] = list("<PII>")
    return "".join(chars)

        ### TODO: SJS 2/3: certain synthetic prospective client names (e.g. Darius) are not automatically redacted

texts = d['text'].astype(str).tolist()
d['text'] = [redact_named_entities(doc) for doc in nlp.pipe(texts)]

# remove stray manual redactions (pilot / Phase 1)

manual_redactions = [
    "[PATIENT NAME]", "[PHONE NUMBER]",
    "[MHP NAME]", "[CITY]", "[URL]",
    ]

for m in manual_redactions:
    d['text'] = d['text'].str.replace(
        m, 
        " ", 
        regex = False,
        )

# remove numerals, newlines

d['text'] = d['text'].str.replace(
    r"[\d\r\n]+", 
    " ", 
    regex = True,
    )

# replace NaN with 0 in `target` varlist

labels = [
    "afrm", "agnt", "brdn", "dmnd", "fitt", 
    "just", "prbl", "rbnd", "refl",
    ]

for l in labels:
    d[l] = pd.to_numeric(d[l], errors = "coerce").fillna(0).astype(int)

d.info()
d.head(6)

<class 'pandas.core.frame.DataFrame'>
Index: 1005 entries, 0 to 1004
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   EmailPairID      1005 non-null   int64 
 1   WithinPatientID  1005 non-null   int64 
 2   FirstInPair      1005 non-null   object
 3   MHP ID           1005 non-null   object
 4   Unique ID        1005 non-null   object
 5   pilot            1005 non-null   int64 
 6   text             1005 non-null   object
 7   prbl             1005 non-null   int64 
 8   agnt             1005 non-null   int64 
 9   afrm             1005 non-null   int64 
 10  brdn             1005 non-null   int64 
 11  dmnd             1005 non-null   int64 
 12  fitt             1005 non-null   int64 
 13  just             1005 non-null   int64 
 14  rbnd             1005 non-null   int64 
 15  refl             1005 non-null   int64 
 16  rtnl             999 non-null    object
 17  note             1003 non-null   objec

,EmailPairID,WithinPatientID,FirstInPair,MHP ID,Unique ID,pilot,text,prbl,agnt,afrm,brdn,dmnd,fitt,just,rbnd,refl,rtnl,note
/;,,,,,,,,,,,,,,,,,,
0,52,5,Y,1086440,52_5_Y_1086440,0,"Hi <|PII|>, Thanks for reaching out! Unfortuna...",0,0,0,0,0,1,1,0,0,<|JUST|> Unfortunately I don’t accept <|PII|> ...,
1,11,4,Y,.,.,1,"Hi <PII>, Thanks for reaching out! What is you...",0,0,0,0,0,0,0,0,0,,
2,316,1,Y,1306860,316_1_Y_1306860,0,"Hello <|PII|>, I received your psychology toda...",0,0,0,0,0,0,0,0,0,,
3,63,5,Y,.,.,1,"Good Morning <PII>, I am very sorry to hear yo...",0,0,1,0,0,1,0,0,1,"<|REFL|> Good Morning Darius, I am very sorry ...",
4,35,10,Y,.,.,1,Hi. This is <PII>. I am returning a call for <...,0,0,0,0,0,0,0,0,0,,
5,12,6,Y,.,.,1,"Hi <PII>, Thank you for reaching out. Before w...",0,0,0,0,0,0,0,0,0,,


In [7]:

# gen `trgt` indicator: dummy code pos(1) rows

d['trgt'] = d[[
    'afrm', 'agnt',
    'fitt', 'refl',
    ]].apply(lambda row: 1 if any(row) else 0, axis = 1)

def augment(df):
    '''
    Detects pos(1) `trgt` rows; duplicates as new row; replaces new row `text` 
    with expert annotator-curated `rtnl` text to augment training instance.
    '''
    new_rows = []
    for index, row in df.iterrows():
        if row['trgt'] > 0:
            new_row = row.copy()
            new_row['text'] = row['rtnl']
            new_rows.append((index + 0.5, new_row))

    if not new_rows:
        return df

    aug_df = pd.DataFrame(
        [row for _, row in new_rows],
        index = [idx for idx, _ in new_rows],
        )
    df = pd.concat([df, aug_df])
    df = df.sort_index(kind = "stable").reset_index(drop = True)
    return df

d = augment(d.copy())

# gen `agmt` indicator: dummy code augmented rows

d['agmt'] = 0
t_indices = d['rtnl'].apply(lambda i: isinstance(i, str))
d.loc[t_indices.shift(1, fill_value = False), 'agmt'] = 1

# remove `rtnl` construct salience delineator pseudoword tokens (parent / Phase 2) / `<PII>` redactions

salience_delineators = [
    "<PII>", "<|PII|>", "<|AFRM|>", "<|AGNT|>", "<|BRDN|>", "<|DMND|>",  
    "<|FITT|>", "<|JUST|>", "<|PRBL|>", "<|RBND|>", "<|REFL|>",
    ]

for s in salience_delineators:
    d['text'] = d['text'].str.replace(
        s, 
        " ", 
        regex = False,
        )

d.shape
d.head(6)

,EmailPairID,WithinPatientID,FirstInPair,MHP ID,Unique ID,pilot,text,prbl,agnt,afrm,brdn,dmnd,fitt,just,rbnd,refl,rtnl,note,trgt,agmt
0,52,5,Y,1086440,52_5_Y_1086440,0,"Hi , Thanks for reaching out! Unfortunately I...",0,0,0,0,0,1,1,0,0,<|JUST|> Unfortunately I don’t accept <|PII|> ...,,1,0
1,52,5,Y,1086440,52_5_Y_1086440,0,Unfortunately I don’t accept and am curren...,0,0,0,0,0,1,1,0,0,<|JUST|> Unfortunately I don’t accept <|PII|> ...,,1,1
2,11,4,Y,.,.,1,"Hi , Thanks for reaching out! What is your in...",0,0,0,0,0,0,0,0,0,,,0,1
3,316,1,Y,1306860,316_1_Y_1306860,0,"Hello , I received your psychology today inqu...",0,0,0,0,0,0,0,0,0,,,0,1
4,63,5,Y,.,.,1,"Good Morning , I am very sorry to hear you ar...",0,0,1,0,0,1,0,0,1,"<|REFL|> Good Morning Darius, I am very sorry ...",,1,1
5,63,5,Y,.,.,1,"Good Morning Darius, I am very sorry to hear...",0,0,1,0,0,1,0,0,1,"<|REFL|> Good Morning Darius, I am very sorry ...",,1,1
